# COPA Demo

In [1]:
import sys, os
from pathlib import Path

if not "cwd" in globals():
   cwd = Path(os.getcwd())
sys.path.insert(0, str(cwd.parents[0]))

In [2]:
TASK_NAME = "COPA"
AUX_TASK_NAME = "SWAG"
BERT_MODEL = "bert-large-cased"

dataloader_config = {
    "batch_size": 4,
    "data_dir": os.environ.get("SUPERGLUEDATA", os.path.join(str(cwd.parents[0]), "data")),
    "splits": ["train", "valid"],
}

trainer_config = {
    "lr": 1e-5,
    "optimizer": "adam",
    "n_epochs": 10,
    "conter_unit": "epochs",
    "evaluation_freq": 0.25,
    "checkpointing": 1,
    "logging": 1,
}

### Train Primary Task from BERT

In [3]:
from dataloaders import get_dataloaders

# Loading primary task data
copa_dataloaders = get_dataloaders(
    task_name=TASK_NAME,
    tokenizer_name=BERT_MODEL,
    **dataloader_config
)

In [4]:
from superglue_tasks import task_funcs

# Defining task
copa_task = task_funcs[TASK_NAME](BERT_MODEL)

In [5]:
from snorkel.mtl.model import MultitaskModel
from snorkel.mtl.trainer import Trainer

copa_model = MultitaskModel(tasks=[copa_task])
trainer = Trainer(**trainer_config)

In [6]:
# Training on COPA an dsaving model -- takes a long time on CPU!
# trainer.train_model(copa_model, copa_dataloaders)
# copa_model.save('best_model_COPA_SuperGLUE_valid_accuracy.pth')

In [7]:
# Alternatively, download and load trained model run ahead of time to save time
# ! wget -nc https://www.dropbox.com/s/c7dv5vgr5lqon61/best_model_COPA_SuperGLUE_valid_accuracy.pth
# copa_model.load('best_model_COPA_SuperGLUE_valid_accuracy.pth')

File ‘best_model_COPA_SuperGLUE_valid_accuracy.pth’ already there; not retrieving.



In [8]:
# Evaluating model
copa_train_loader, copa_dev_loader = copa_dataloaders
copa_score = copa_model.score(copa_dev_loader)
print(f"COPA (from BERT) Accuracy: {copa_score['COPA/SuperGLUE/valid/accuracy']}")

COPA (from BERT) Accuracy: 0.72


### Train Auxiliary Task (SWAG) from BERT

In [9]:
# Getting dataloaders, task, and model for SWAG
# Note: this is a large dataset—it takes a few minutes to load

swag_dataloaders = get_dataloaders(
    task_name=AUX_TASK_NAME,
    tokenizer_name=BERT_MODEL,
    **dataloader_config
)

swag_task = task_funcs[AUX_TASK_NAME](BERT_MODEL)
swag_model = MultitaskModel(tasks=[swag_task])

In [10]:
# Training on SWAG and saving model -- takes a long time on CPU!
# trainer.train_model(swag_model, swag_dataloaders)
# swag_model.save('./best_model_SWAG_SuperGLUE.pth')

### Train Primary Task from BERT + SWAG

In [11]:
from copy import deepcopy

# Initialize COPA model with weights from trained SWAG model
copa_swag_model = deepcopy(copa_model)
# copa_swag_model.load('best_model_SWAG_SuperGLUE.pth')

In [12]:
from snorkel.mtl.trainer import Trainer

# Training on COPA and saving model -- takes a long time on CPU!
# trainer.train_model(copa_swag_model, copa_dataloaders)
# copa_swag_model.save('best_model_COPA_SWAG_AUX_SuperGLUE_valid_accuracy.pth')

In [13]:
# Alternatively, download and load trained model run ahead of time to save time
# ! wget -nc https://www.dropbox.com/s/xj2zubij3vqtge5/best_model_COPA_SWAG_AUX_SuperGLUE_valid_accuracy.pth
# copa_swag_model.load('best_model_COPA_SWAG_AUX_SuperGLUE_valid_accuracy.pth')

File ‘best_model_COPA_SWAG_AUX_SuperGLUE_valid_accuracy.pth’ already there; not retrieving.



In [14]:
copa_swag_score = copa_swag_model.score(copa_dev_loader)

### Compare performance with/without Aux. Task

In [15]:
print(f"COPA (from BERT) Accuracy: {copa_score['COPA/SuperGLUE/valid/accuracy']}")
print(f"COPA (from BERT + SWAG) Accuracy: {copa_swag_score['COPA/SuperGLUE/valid/accuracy']}")

COPA (from BERT) Accuracy: 0.72
COPA (from BERT + SWAG) Accuracy: 0.76
